# Hands-On Assignment 5

In this assignment, you will practice everything that you have learned so far in an end-to-end setting.
You will be provided with a dataset that is **unique to you**, and your task is to perform
all the steps from previous assignments to clean, explore, visualize, and analyze your dataset.

**Written Portion**: Additionally, you will create a report that describes your process and provides insights about your dataset.
Each section that should appear in your report is noted with an orange star (like normal HO tasks).  The report should be  4-6 pages (12 pt font, 1.5 line spacing), and turned in on Canvas as a PDF.

The coding aspect for this assignment will be turned in the same was as all other HO's,
by submitting this file to the autograder.


For this assignment, feel free to make additional functions instead of implementing everything in the provided function.

The objective of this assignment is for you to apply and solidify the skills you have learned in previous assignments.

# Prompt

You have graduated from this class, and are a huge success!
You landed a job doing data science at some fancy company.

You just got a new client with some really interesting problems you get to solve.
Unfortunately, because of a big mess-up on their side the data's metadata got corrupted up
(and the person that used to maintain the data just took a vow of silence and moved to a bog).

The only column you are sure about is the `label` column,
which contains a numeric label for each row.
Aside from that, the client does not know anything about the names, content, or even data types for each column.

Your task is to explore, clean, and analyze this data.
You should have already received an email with the details on obtaining your unique data.
Place it in the same directory as this notebook (and your `local_grader.py` script) and name it `data.txt`.

*I know this prompt may sound unrealistic, but I have literally been in a situation exactly like this.
I was working at a database startup, and one of our clients gave us data with over 70 columns and more than a million records and told us:
"The person who used to manage the data is no longer working with us, but this was the data they used to make all their decisions.
We also lost all the metadata information, like column names."
...
Working in industry is not always glamorous.
-Eriq*

# Part 0: Explore Your Data

Before you start doing things to/with your data, it's always a good idea to load up your data and take a look.

In [2]:
import pandas
import re
import sklearn
import numpy

# Modify this to point to your data.
unique_data = pandas.read_csv('data.txt', sep = "\t")
unique_data

FileNotFoundError: [Errno 2] No such file or directory: 'data.txt'

Don't forget to checkout the column information.

In [ ]:
unique_data.info()

And any numeric information.

In [ ]:
unique_data.describe()

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Introduction</h4>

Briefly describe the dataset you’re given and define the goal of the project and how you approach it.
For example, you can present a basic introduction of your data (shape and proposed data types)
and your goal is to use these features to predict the label of the response variable.
Then you propose a few models that are suitable for this project which will be introduced in the modeling section.

# Part 1: Data Cleaning

As always, we should start with data cleaning.
Take what you learned from HO3 to clean up this messy data to a point where it is ready for machine learning algorithms.

Some things you may want to do:
 - Deal with missing/empty values.
 - Fix numeric columns so that they actually contain numbers.
 - Remove inconsistencies from columns.
 - Assign a data type to each column.

<h4 style="color: darkorange; font-size: x-large";>★ Task 1.A</h4>

Complete the following function that takes in a DataFrame and outputs a clean version of the DataFrame.
You can assume that the frame has all the same structure as your unique dataset.
You can return the same or a new data frame.

In [13]:
def clean_data(frame):
    new_frame = frame.copy()
    ignore_columns = []
    for i in range(len(frame.columns)):
        column = frame.columns[i]
        column_values = frame[column]
        for j, value in column_values.items():
            if value != numpy.nan and value != "?":
                value = str(value)
                if "." in value:
                    cleanval = re.match(r"-?\d+\.\d+", str(value))
                    if cleanval is not None:
                        break
                else:
                    cleanval = re.match(r"-?\d+", str(value))
                    if cleanval is not None:
                        break   
                    else:
                        ignore_columns.append(column)
                        break
    for col in new_frame.columns:
        if col not in ignore_columns:
            for index, number in enumerate(new_frame[col]):
                number = str(number).strip()
                if not number:
                    new_frame.at[index, col] = numpy.nan
                    continue
                numberfound = False
                lindex = number.find("(")
                rindex = number.find(")")
                if lindex != -1 and rindex != -1:
                    number = number[:lindex] + number[rindex + 1:]
                
                if '.' in number:
                    cleanval = re.search(r"-?\d+\.\d+", number)
                    if cleanval is not None:
                        if len(cleanval.group(0)) > 0:
                            cleanval = float((cleanval.group(0)))
                            new_frame.at[index, col] = float(cleanval)
                            numberfound = True
                elif "." not in number:
                    cleanval = re.search(r"-?\d+", number)
                    if cleanval is not None:
                        cleanval = cleanval.group(0)
                        if len(cleanval) > 0:
                            new_frame.at[index, col] = int(cleanval)
                            numberfound = True
                if not numberfound:
                    new_frame.at[index, col] = numpy.nan
    
    for column in new_frame.columns:
        column_values = new_frame[column].dropna()
        if all(isinstance(value, int) for value in column_values):
            new_frame[column] = column_values.astype(int)
        elif any(isinstance(value, float) for value in column_values):
            new_frame[column] = column_values.astype(float)
        else:
            new_frame[column] = column_values.astype(str)
    return new_frame

unique_data = clean_data(unique_data)
unique_data

,label,col_00,col_01,col_02,col_03,col_04,col_05,col_06,col_07,col_08,col_09,col_10,col_11,col_12
0,2,1250.0,0.9100,Oakland,Volleyball,-1.1242,1.5959,1200.0,1274.0,1257.0,0.1069,1521.0,biotechnology,721.0
1,5,451.0,-0.8472,Long Beach,running,0.6164,0.4964,-1328.0,329.0,-474.0,1.4402,1020.0,Electrical Engineering,-507.0
2,0,-1232.0,1.6729,Riverside,ice hockey,1.0341,0.1537,-376.0,1055.0,1052.0,0.6991,-765.0,computational media,124.0
3,2,702.0,-0.0621,Stockton,"boxing, baseball",-0.4193,0.4083,746.0,1549.0,1844.0,0.5507,1896.0,Technology and Information Management,870.0
4,0,-339.0,1.4487,Chula Vista,ice hockey,0.6834,0.1259,326.0,1260.0,200.0,2.4040,1900.0,Applied Mathematics,932.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593,3,629.0,0.3127,Los Angeles,basketball,1.9682,1.2458,-1934.0,1445.0,-383.0,0.4067,618.0,Technology and Information Management,-767.0
1594,5,537.0,NaN,Long Beach,running,1.0070,0.4218,799.0,-749.0,-531.0,1.4479,43.0,Natural Language Processing,873.0
1595,4,35.0,0.8890,Fresno,track & field,0.8181,1.2241,2073.0,1661.0,1643.0,-0.3686,1464.0,Human Computer Interaction,79.0
1596,5,1681.0,-0.2493,Long Beach,running,1.6034,-0.3015,593.0,-1296.0,-467.0,0.2499,-232.0,Bioengineering,-467.0


Now we should also be able to view all the numeric columns.

In [14]:
unique_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1598 entries, 0 to 1597
Data columns (total 14 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   label   1598 non-null   int64  
 1   col_00  1581 non-null   float64
 2   col_01  1580 non-null   float64
 3   col_02  1586 non-null   object 
 4   col_03  1586 non-null   object 
 5   col_04  1582 non-null   float64
 6   col_05  1580 non-null   float64
 7   col_06  1587 non-null   float64
 8   col_07  1589 non-null   float64
 9   col_08  1585 non-null   float64
 10  col_09  1586 non-null   float64
 11  col_10  1582 non-null   float64
 12  col_11  1584 non-null   object 
 13  col_12  1581 non-null   float64
dtypes: float64(10), int64(1), object(3)
memory usage: 174.9+ KB


<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Cleaning</h4>

Describe the steps you took for data cleaning.
Why did you do this?
Did you have to make some choices along the way? If so, describe them.

# Part 2: Data Visualization

Once you have cleaned up the data, it is time to explore it and find interesting things.
Part of this exploration, will be visualizing the data in a way that makes it easier for yourself and others to understand.
Use what you have learned in HO1 and HO2 to create some visualizations for your dataset.

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Data Visualization</h4>

Create at least two different visualizations that help describe what you see in your dataset.
Include these visualizations in your report along with descriptions of
how you created the visualization,
what data preparation you had to do for the visualization (aside from the data cleaning in the previous part),
and what the visualization tells us about the data.

In [1]:
def plot_infection_probability_vs_titer(frame):
    # Get all the possible titer values in a list.
    titer_values = list(range(frame['col_00'].min(), frame['col_00'].max() + 1))
    infected_rate = []

    for titer_value in titer_values:
        infected_count = len(frame[(frame['col_00'] == titer_value) & frame['label']])
        count = len(frame[(frame['col_00'] == titer_value)])

        if (count == 0):
            # Matplotlib will ignore None/NaN values.
            infected_rate.append(None)
        else:
            infected_rate.append(infected_count / count)

    matplotlib.pyplot.plot(infected_rate)
    matplotlib.pyplot.title('Sample Frequency of Label vs Col_00 Value')
    matplotlib.pyplot.xlabel('Col_00')
    matplotlib.pyplot.ylabel('Label')

plot_infection_probability_vs_titer(unique_data)

NameError: name 'unique_data' is not defined

# Part 3: Modeling

Now that you have a good grasp of your clean data,
it is time to do some machine learning!
(Technically all our previous steps were also machine learning,
but now we get to use classifiers!)

Use the skills you developed to select **three** classifiers and implement them on your data.
For example, you can narrow down your choices to three classifiers which may include:
- Logistic regression
- K-nearest neighbors
- Decision tree
- Or others

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.A</h4>

Complete the following function that takes in no parameters,
and returns a list with **three** untrained classifiers you are going to explore in this assignment.
This method may set parameters/options for the classifiers, but should not do any training/fitting.

For example, if you wanted to use logistic regression,
then **one** of your list items may be:
```
sklearn.linear_model.LogisticRegression()
```

In [18]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

def create_classifiers():
    classifiers = []

    classifiers.append(DecisionTreeClassifier())
    classifiers.append(LogisticRegression())
    classifiers.append(KNeighborsClassifier(n_neighbors=3))
    
    return classifiers

my_classifiers = create_classifiers()
my_classifiers


[DecisionTreeClassifier(),
 LogisticRegression(),
 KNeighborsClassifier(n_neighbors=3)]

Now that we have some classifiers, we can see how they perform.

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.B</h4>

Complete the following function that takes in an untrained classifier, a DataFrame, and a number of folds.
This function should run k-fold cross validation with the classifier and the data,
and return a list with the accuracy of each run of cross validation.
You can assume that the frame has the column `label` and the rest of the columns can be considered clean numeric features.

Note that you may have to break your frame into features and labels to do this.
Do not change the passed-in frame (make copies instead).

If you are getting any `ConvergenceWarning`s you may either ignore them,
or try and address them
(they will not affect your autograder score, but may be something to discuss in the written portion of this assignment).

In [ ]:
def cross_fold_validation(classifier, frame, folds):
    return NotImplemented

my_classifiers_scores = []
for classifier in my_classifiers:
    accuracy_scores = cross_fold_validation(classifier, unique_data, 5)
    my_classifiers_scores.append(accuracy_scores)
    print("Classifier: %s, Accuracy: %s." % (type(classifier).__name__, accuracy_scores))

<h4 style="color: darkorange; font-size: x-large";>★ Task 3.C</h4>

Complete the following function that takes in two equally-sized lists of numbers and a p-value.
This function should compute whether there is a statistical significance between
these two lists of numbers using a [Student's t-test](https://en.wikipedia.org/wiki/Student%27s_t-test)
at the given p-value.
Return `True` if there is a statistical significance, and `False` otherwise.
Hint: If you wish, you may use the `ttest_ind()` [method](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html) provided in the scipy package. 

In [ ]:
def significance_test(a_values, b_values, p_value):
    return NotImplemented

for i in range(len(my_classifiers)):
    for j in range(i + 1, len(my_classifiers)):
        significant = significance_test(my_classifiers_scores[i], my_classifiers_scores[j], 0.10)
        print("%s vs %s: %s" % (type(my_classifiers[i]).__name__,
                                type(my_classifiers[j]).__name__, significant))

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Modeling</h4>

Describe the classifiers you have chosen.
Be sure to include all details about any parameter settings used for the algorithms.

Compare the performance of your models using k-fold validation.
You may look at accuracy, F1 or other measures.

Then, briefly summarize your results.
Are your results statistically significant?
Is there a clear winner?
What do the standard deviations look like, and what do they tell us about the different models?
Include a table like Table 1.

<center>Table 1: Every table need a caption.</center>

| Model | Mean Accuracy | Standard Deviation of Accuracy |
|-------|---------------|--------------------------------|
| Logistic Regression | 0.724 | 0.004
| K-Nearest Neighbor | 0.750 | 0.003
| Decision Tree | 0.655 | 0.011

# Part 4: Analysis

Now, take some time to go over your results for each classifier and try to make sense of them.
 - Why do some classifiers work better than others?
 - Would another evaluation metric work better than vanilla accuracy?
 - Is there still a problem in the data that should fixed in data cleaning?
 - Does the statistical significance between the different classifiers make sense?
 - Are there parameters for the classifier that I can tweak to get better performance?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Analysis</h4>

Discuss your observations, the relationship you found, and how you applied concepts from the class to this project.
For example, you may find that some feature has the most impact in predicting your response variable or removing a feature improves the model accuracy.
Or you may observe that your training accuracy is much higher than your test accuracy and you may want to explain what issues may arise.

# Part 5: Conclusion

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: Conclusion</h4>

Briefly summarize the important results and conclusions presented in the project.
What are the important points illustrated by your work?
Are there any areas for further investigation or improvement?

<h4 style="color: darkorange; font-size: x-large";>★ Written Task: References</h4>

Include a standard bibliography with citations referring to techniques or published papers you used throughout your report (if you used any).

For example:
```
[1] Derpanopoulos, G. (n.d.). Bayesian Model Checking & Comparison.
https://georgederpa.github.io/teaching/modelChecking.html.
```

# Part XC: Extra Credit

So far you have used a synthetic dataset that was created just for you.
But, data science is always more interesting when you are dealing with actual data from the real world.
Therefore, you will have an opportunity for extra credit on this assignment using real-world data.

For extra credit, repeat the **written tasks** of Parts 0 through 4 with an additional dataset that you find yourself.
For the written portion of the extra credit for Part 0, include information about where you got the data and what the data represents.
You may choose any dataset that represents real data (i.e., is **not** synthetic or generated)
and is **not** [pre-packaged in scikit-learn](https://scikit-learn.org/stable/datasets.html).

Below are some of the many places you can start looking for datasets:
 - [Kaggle](https://www.kaggle.com/datasets) -- Kaggle is a website focused around machine learning competitions,
       where people compete to see who can get the best results on a dataset.
       It is very popular in the machine learning community and has thousands of datasets with descriptions.
       Make sure to read the dataset's description, as Kaggle also has synthetic datasets.
 - [data.gov](https://data.gov/) -- A portal for data from the US government.
        The US government has a lot of data, and much of it has to be available to the public by law.
        This portal contains some of the more organized data from several different government agencies.
        In general, the government has A LOT of interesting data.
        It may not always be clean (remember the CIA factbook), but it is interesting and available.
        All data here should be real-world, but make sure to read the description to verify.
 - [UCI's Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets.php) -- UC Irvine has their own data repository with a few hundred datasets on many different topics.
        Make sure to read the dataset's description, as UCI also has synthetic datasets.
 - [WHO's Global Health Observatory](https://apps.who.int/gho/data/node.home) -- The World Health Organization keeps track of many different health-related statistics for most of the countries in the world.
        All data here should be real-world, but make sure to read the description to verify.
 - [Google's Dataset Search](https://datasetsearch.research.google.com/) -- Google indexes many datasets that can be searched here.

You can even create a dataset from scratch if you find some data you like that is not already organized into a specific dataset.
The only real distinction between "data" and a "dataset" is that a dataset is organized and finite (has a fixed size).

Create a new section in your written report for this extra credit and include all the written tasks for the extra credit there.
Each written task/section that you complete for your new dataset is eligible for extra credit (so you can still receive some extra credit even if you do not complete all parts).
There is no need to submit any code for the extra credit.
If you created a new dataset, include the dataset or links to it with your submission.